## ML Predict credict card 

### import Libray

In [100]:
#import library
import sklearn
import warnings
import matplotlib
%matplotlib inline
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from mlxtend.classifier import StackingCVClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

### Load data from Trian

In [32]:
# load data
df = pd.read_excel('train.xlsx')
df.head()
len(df)

14995

#### we will delete the collumn id cuz it not relevant to our prediction

In [33]:
# drop id column
df.drop(columns=['ID'],axis=1,inplace=True)

In [34]:
df

,account_type,gender,age,region_code,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,...,debit_count_may,max_credit_amount_may,debit_amount_jun,credit_amount_jun,credit_count_jun,debit_count_jun,max_credit_amount_jun,loan_enq,emi_active,cc_cons
0,current,M,35,708,24893.00,378.00,10288.00,29664.00,16291.400,11432.00,...,40,24563.0,30051.32,12761.00,2,65,50836.00,Y,1674.09,20014
1,current,M,35,123,18941.62,966.00,20672.00,287.00,4217.000,885.00,...,78,23948.0,42119.05,76206.00,21,63,23226.00,Y,13043.34,10173
2,current,M,55,802,5678.87,2724.00,1964.50,3933.11,23956.250,5168.00,...,82,10752.0,58373.25,75283.00,7,1,27036.00,Y,25375.27,16095
3,current,F,29,867,30489.50,1236.00,12609.88,9138.14,17521.000,13650.30,...,38,12607.0,69851.51,68708.00,21,83,43037.00,Y,3544.33,7707
4,current,M,27,958,2328.22,1597.54,19979.75,1045.85,11686.255,628.00,...,2,21440.0,52292.68,199925.50,2,35,17284.00,Y,5026.50,130263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14990,current,F,53,653,10443.76,1218.85,7252.24,3873.00,6633.000,10107.00,...,8,6936.0,487751.19,42625.50,14,8,82851.00,Y,78966.76,4263
14991,current,M,44,535,29890.33,6308.00,16025.50,7610.00,2213.080,841.60,...,20,202639.0,8666.01,34987.55,11,18,118110.47,Y,4762.10,11019
14992,current,M,37,464,4148.98,948.74,19360.26,494.00,44363.910,1801.77,...,5,44421.0,97896.48,261432.00,4,73,49545.00,Y,2877.33,404227
14993,current,M,220,536,115574.00,7191.05,44341.13,34817.20,20679.000,1010.00,...,25,22409.0,22693.92,33025.29,3,80,16130.00,Y,13006.84,28813


#### checking missing value

In [35]:
round(df.isnull().sum())

account_type             0
gender                   0
age                      0
region_code              0
cc_cons_apr              0
dc_cons_apr              0
cc_cons_may              0
dc_cons_may              0
cc_cons_jun              0
dc_cons_jun              0
cc_count_apr             0
cc_count_may             0
cc_count_jun             0
dc_count_apr             0
dc_count_may             0
dc_count_jun             0
card_lim                 0
personal_loan_active     0
vehicle_loan_active      0
personal_loan_closed     0
vehicle_loan_closed      0
investment_1             0
investment_2             0
investment_3             0
investment_4             0
debit_amount_apr         0
credit_amount_apr        0
debit_count_apr          0
credit_count_apr         0
max_credit_amount_apr    0
debit_amount_may         0
credit_amount_may        0
credit_count_may         0
debit_count_may          0
max_credit_amount_may    0
debit_amount_jun         0
credit_amount_jun        0
c

#### Split catepory for non-numeric col

In [37]:
# change the datatypes of the columns accordingly for generating dummies
categorical_columns=['account_type','gender','region_code','personal_loan_active',
                     'vehicle_loan_active','personal_loan_closed','vehicle_loan_closed','loan_enq']
for c in categorical_columns:
    df[c]=df[c].astype('category');

In [42]:
df

,account_type,gender,age,region_code,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,...,debit_count_may,max_credit_amount_may,debit_amount_jun,credit_amount_jun,credit_count_jun,debit_count_jun,max_credit_amount_jun,loan_enq,emi_active,cc_cons
0,current,M,35,708,24893.00,378.00,10288.00,29664.00,16291.400,11432.00,...,40,24563.0,30051.32,12761.00,2,65,50836.00,Y,1674.09,20014
1,current,M,35,123,18941.62,966.00,20672.00,287.00,4217.000,885.00,...,78,23948.0,42119.05,76206.00,21,63,23226.00,Y,13043.34,10173
2,current,M,55,802,5678.87,2724.00,1964.50,3933.11,23956.250,5168.00,...,82,10752.0,58373.25,75283.00,7,1,27036.00,Y,25375.27,16095
3,current,F,29,867,30489.50,1236.00,12609.88,9138.14,17521.000,13650.30,...,38,12607.0,69851.51,68708.00,21,83,43037.00,Y,3544.33,7707
4,current,M,27,958,2328.22,1597.54,19979.75,1045.85,11686.255,628.00,...,2,21440.0,52292.68,199925.50,2,35,17284.00,Y,5026.50,130263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14990,current,F,53,653,10443.76,1218.85,7252.24,3873.00,6633.000,10107.00,...,8,6936.0,487751.19,42625.50,14,8,82851.00,Y,78966.76,4263
14991,current,M,44,535,29890.33,6308.00,16025.50,7610.00,2213.080,841.60,...,20,202639.0,8666.01,34987.55,11,18,118110.47,Y,4762.10,11019
14992,current,M,37,464,4148.98,948.74,19360.26,494.00,44363.910,1801.77,...,5,44421.0,97896.48,261432.00,4,73,49545.00,Y,2877.33,404227
14993,current,M,220,536,115574.00,7191.05,44341.13,34817.20,20679.000,1010.00,...,25,22409.0,22693.92,33025.29,3,80,16130.00,Y,13006.84,28813


### Feature Engineering

In [43]:
# split df into x and y for train_test_split
df_X=df.drop(columns=['cc_cons','region_code'])
df_X.head()

,account_type,gender,age,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,cc_count_apr,...,credit_count_may,debit_count_may,max_credit_amount_may,debit_amount_jun,credit_amount_jun,credit_count_jun,debit_count_jun,max_credit_amount_jun,loan_enq,emi_active
0,current,M,35,24893.00,378.00,10288.00,29664.00,16291.400,11432.0,2,...,2,40,24563.0,30051.32,12761.0,2,65,50836.0,Y,1674.09
1,current,M,35,18941.62,966.00,20672.00,287.00,4217.000,885.0,1,...,2,78,23948.0,42119.05,76206.0,21,63,23226.0,Y,13043.34
2,current,M,55,5678.87,2724.00,1964.50,3933.11,23956.250,5168.0,43,...,2,82,10752.0,58373.25,75283.0,7,1,27036.0,Y,25375.27
3,current,F,29,30489.50,1236.00,12609.88,9138.14,17521.000,13650.3,53,...,23,38,12607.0,69851.51,68708.0,21,83,43037.0,Y,3544.33
4,current,M,27,2328.22,1597.54,19979.75,1045.85,11686.255,628.0,6,...,43,2,21440.0,52292.68,199925.5,2,35,17284.0,Y,5026.50


In [44]:
df_Y=df['cc_cons']
df_Y.head()

0     20014
1     10173
2     16095
3      7707
4    130263
Name: cc_cons, dtype: int64

In [45]:
# create calculated fields
# Total dc and cc cons for 3 months
df_X['cc_cons_sum']= df_X['cc_cons_apr'] + df_X['cc_cons_may'] + df_X['cc_cons_jun']
df_X['dc_cons_sum']= df_X['dc_cons_apr']+ df_X['dc_cons_may'] + df_X['dc_cons_jun']
# Total dc and cc count for 3 months
df_X['cc_count_sum'] = df_X['cc_count_apr'] + df_X['cc_count_may'] + df_X['cc_count_jun']
df_X['dc_count_sum'] = df_X['dc_count_apr'] + df_X['dc_count_may'] + df_X['dc_count_jun']
# Total dc and cc cons average for 3 months
df_X['cc_cons_avg'] = 0
df_X.loc[df_X.cc_count_sum != 0,'cc_cons_avg'] = df_X.loc[df_X.cc_count_sum != 0,'cc_cons_sum']/df_X.loc[df_X.cc_count_sum != 0,'cc_count_sum']
df_X['dc_cons_avg'] = 0
df_X.loc[df_X.dc_count_sum != 0,'dc_cons_avg'] = df_X.loc[df_X.dc_count_sum != 0,'dc_cons_sum']/df_X.loc[df_X.dc_count_sum != 0,'dc_count_sum']
# Average dc cons for each month
df_X['dc_cons_apr_avg']= 0
df_X.loc[df_X.dc_count_apr != 0,'dc_cons_apr_avg'] = df_X.loc[df_X.dc_count_apr != 0,'dc_cons_apr']/df_X.loc[df_X.dc_count_apr != 0,'dc_count_apr']
df_X['dc_cons_may_avg'] = 0
df_X.loc[df_X.dc_count_may != 0,'dc_cons_may_avg'] = df_X.loc[df_X.dc_count_may != 0,'dc_cons_may']/df_X.loc[df_X.dc_count_may != 0,'dc_count_may']
df_X['dc_cons_jun_avg']= 0
df_X.loc[df_X.dc_count_jun != 0,'dc_cons_jun_avg'] = df_X.loc[df_X.dc_count_jun != 0,'dc_cons_jun']/df_X.loc[df_X.dc_count_jun != 0,'dc_count_jun']
# Average cc cons for each month
df_X['cc_cons_apr_avg'] = 0
df_X.loc[df_X.cc_count_apr != 0,'cc_cons_apr_avg'] = df_X.loc[df_X.cc_count_apr != 0,'cc_cons_apr']/df_X.loc[df_X.cc_count_apr != 0,'cc_count_apr']
df_X['cc_cons_may_avg'] = 0
df_X.loc[df_X.cc_count_may != 0,'cc_cons_may_avg'] = df_X.loc[df_X.cc_count_may != 0,'cc_cons_may']/df_X.loc[df_X.cc_count_may != 0,'cc_count_may']
df_X['cc_cons_jun_avg'] = 0
df_X.loc[df_X.cc_count_jun != 0,'cc_cons_jun_avg'] = df_X.loc[df_X.cc_count_jun != 0,'cc_cons_jun']/df_X.loc[df_X.cc_count_jun != 0,'cc_count_jun']

In [49]:
# Total credit and debit amount for 3 months
df_X['credit_amount_sum'] = df_X.credit_amount_apr + df_X.credit_amount_may + df_X.credit_amount_jun
df_X['debit_amount_sum'] = df_X.debit_amount_apr + df_X.debit_amount_may + df_X.debit_amount_jun
# Total credit and debit count for 3 months
df_X['credit_count_sum'] = df_X.credit_count_apr + df_X.credit_count_may + df_X.credit_count_jun
df_X['debit_count_sum'] = df_X.debit_count_apr + df_X.debit_count_may + df_X.debit_count_jun
# Total credit and debit amount average for 3 months
df_X['credit_amount_avg'] = 0
df_X.loc[df_X.credit_count_sum != 0,'credit_amount_avg'] = df_X.loc[df_X.credit_count_sum != 0,'credit_amount_sum']/df_X.loc[df_X.credit_count_sum != 0,'credit_count_sum']
df_X['debit_amount_avg'] = 0
df_X.loc[df_X.debit_count_sum != 0,'debit_amount_avg'] = df_X.loc[df_X.debit_count_sum != 0,'debit_amount_sum']/df_X.loc[df_X.debit_count_sum != 0,'debit_count_sum']
# average debit amount for each month
df_X['debit_amount_apr_avg']= 0
df_X.loc[df_X.debit_count_apr != 0,'debit_amount_apr_avg'] = df_X.loc[df_X.debit_count_apr != 0,'debit_amount_apr']/df_X.loc[df_X.debit_count_apr != 0,'debit_count_apr']
df_X['debit_amount_may_avg'] = 0
df_X.loc[df_X.debit_count_may != 0,'debit_amount_may_avg'] = df_X.loc[df_X.debit_count_may != 0,'debit_amount_may']/df_X.loc[df_X.debit_count_may != 0,'debit_count_may']
df_X['debit_amount_jun_avg'] = 0
df_X.loc[df_X.debit_count_jun != 0,'debit_amount_jun_avg'] = df_X.loc[df_X.debit_count_jun != 0,'debit_amount_jun']/df_X.loc[df_X.debit_count_jun != 0,'debit_count_jun']
# average credit amount for each month
df_X['credit_amount_apr_avg'] = 0
df_X.loc[df_X.credit_count_apr != 0,'credit_amount_apr_avg'] = df_X.loc[df_X.credit_count_apr != 0,'credit_amount_apr']/df_X.loc[df_X.credit_count_apr != 0,'credit_count_apr']
df_X['credit_amount_may_avg'] = 0
df_X.loc[df_X.credit_count_may != 0,'credit_amount_may_avg'] = df_X.loc[df_X.credit_count_may != 0,'credit_amount_may']/df_X.loc[df_X.credit_count_may != 0,'credit_count_may']
df_X['credit_amount_jun_avg'] = 0
df_X.loc[df_X.credit_count_jun != 0,'credit_amount_jun_avg'] = df_X.loc[df_X.credit_count_jun != 0,'credit_amount_jun']/df_X.loc[df_X.credit_count_jun != 0,'credit_count_jun']

In [50]:
# percentage credit amount per month
df_X['perc_credit_amount_apr'] = 0
df_X.loc[df_X.max_credit_amount_apr!= 0,'perc_credit_amount_apr'] = df_X.loc[df_X.max_credit_amount_apr!= 0,'credit_amount_apr']/df_X.loc[df_X.max_credit_amount_apr!= 0,'max_credit_amount_apr']
df_X['perc_credit_amount_may'] = 0
df_X.loc[df_X.max_credit_amount_may!= 0,'perc_credit_amount_may'] = df_X.loc[df_X.max_credit_amount_may!= 0,'credit_amount_may']/df_X.loc[df_X.max_credit_amount_may!= 0,'max_credit_amount_may']
df_X['perc_credit_amount_jun'] = 0
df_X.loc[df_X.max_credit_amount_jun!= 0,'perc_credit_amount_jun'] = df_X.loc[df_X.max_credit_amount_jun!= 0,'credit_amount_jun']/df_X.loc[df_X.max_credit_amount_jun!= 0,'max_credit_amount_jun']

In [51]:
# proportion of debit and credit consumptions per month
df_X['cc_cons_apr_prop'] = 0
df_X.loc[df_X.cc_cons_sum!= 0,'cc_cons_apr_prop'] = df_X.loc[df_X.cc_cons_sum!= 0,'cc_cons_apr']/df_X.loc[df_X.cc_cons_sum != 0,'cc_cons_sum']
df_X['cc_cons_may_prop'] = 0
df_X.loc[df_X.cc_cons_sum != 0,'cc_cons_may_prop'] = df_X.loc[df_X.cc_cons_sum != 0,'cc_cons_may']/df_X.loc[df_X.cc_cons_sum != 0,'cc_cons_sum']
df_X['cc_cons_jun_prop'] = 0
df_X.loc[df_X.cc_cons_sum != 0,'cc_cons_jun_prop'] = df_X.loc[df_X.cc_cons_sum != 0,'cc_cons_jun']/df_X.loc[df_X.cc_cons_sum != 0,'cc_cons_sum']
df_X['dc_cons_apr_prop'] = 0
df_X.loc[df_X.dc_cons_sum != 0,'dc_cons_apr_prop'] = df_X.loc[df_X.dc_cons_sum != 0,'dc_cons_apr']/df_X.loc[df_X.dc_cons_sum != 0,'dc_cons_sum']
df_X['dc_cons_may_prop'] = 0
df_X.loc[df_X.dc_cons_sum != 0,'dc_cons_may_prop'] = df_X.loc[df_X.dc_cons_sum != 0,'dc_cons_may']/df_X.loc[df_X.dc_cons_sum != 0,'dc_cons_sum']
df_X['dc_cons_jun_prop'] = 0
df_X.loc[df_X.dc_cons_sum != 0,'dc_cons_jun_prop'] = df_X.loc[df_X.dc_cons_sum != 0,'dc_cons_jun']/df_X.loc[df_X.dc_cons_sum != 0,'dc_cons_sum']

In [52]:
# proportion of debit and credit amounts per month
df_X['credit_amount_apr_prop'] = 0
df_X.loc[df_X.credit_amount_sum != 0,'credit_amount_apr_prop'] = df_X.loc[df_X.credit_amount_sum != 0,'credit_amount_apr']/df_X.loc[df_X.credit_amount_sum != 0,'credit_amount_sum']
df_X['credit_amount_may_prop'] = 0
df_X.loc[df_X.credit_amount_sum != 0,'credit_amount_may_prop'] = df_X.loc[df_X.credit_amount_sum != 0,'credit_amount_may']/df_X.loc[df_X.credit_amount_sum != 0,'credit_amount_sum']
df_X['credit_amount_jun_prop'] = 0
df_X.loc[df_X.credit_amount_sum != 0,'credit_amount_jun_prop'] = df_X.loc[df_X.credit_amount_sum != 0,'credit_amount_jun']/df_X.loc[df_X.credit_amount_sum != 0,'credit_amount_sum']
df_X['debit_amount_apr_prop'] = 0
df_X.loc[df_X.debit_amount_sum != 0,'debit_amount_apr_prop'] = df_X.loc[df_X.debit_amount_sum != 0,'debit_amount_apr']/df_X.loc[df_X.debit_amount_sum != 0,'debit_amount_sum']
df_X['debit_amount_may_prop'] = 0
df_X.loc[df_X.debit_amount_sum != 0,'debit_amount_may_prop'] = df_X.loc[df_X.debit_amount_sum != 0,'debit_amount_may']/df_X.loc[df_X.debit_amount_sum != 0,'debit_amount_sum']
df_X['debit_amount_jun_prop'] = 0
df_X.loc[df_X.debit_amount_sum != 0,'debit_amount_jun_prop'] = df_X.loc[df_X.debit_amount_sum != 0,'debit_amount_jun']/df_X.loc[df_X.debit_amount_sum != 0,'debit_amount_sum']

In [ ]:
# debit by credit cons and amount ratios
df_X['d_by_c_cons_ratio_apr'] = 0
df_X.loc[df_X.cc_cons_apr != 0,'d_by_c_cons_ratio_apr'] = df_X.loc[df_X.cc_cons_apr != 0,'dc_cons_apr']/df_X.loc[df_X.cc_cons_apr != 0,'cc_cons_apr']
df_X['d_by_c_cons_ratio_may'] = 0
df_X.loc[df_X.cc_cons_may != 0,'d_by_c_cons_ratio_may'] = df_X.loc[df_X.cc_cons_may != 0,'dc_cons_may']/df_X.loc[df_X.cc_cons_may != 0,'cc_cons_may']
df_X['d_by_c_cons_ratio_jun'] = 0
df_X.loc[df_X.cc_cons_jun != 0,'d_by_c_cons_ratio_jun'] = df_X.loc[df_X.cc_cons_jun != 0,'dc_cons_jun']/df_X.loc[df_X.cc_cons_jun != 0,'cc_cons_jun']
df_X['d_by_c_amount_ratio_apr'] = 0
df_X.loc[df_X.credit_amount_apr != 0,'d_by_c_amount_ratio_apr'] = df_X.loc[df_X.credit_amount_apr != 0,'debit_amount_apr']/df_X.loc[df_X.credit_amount_apr != 0,'credit_amount_apr']
df_X['d_by_c_amount_ratio_may'] = 0
df_X.loc[df_X.credit_amount_may != 0,'d_by_c_amount_ratio_may'] = df_X.loc[df_X.credit_amount_may != 0,'debit_amount_may']/df_X.loc[df_X.credit_amount_may != 0,'credit_amount_may']
df_X['d_by_c_amount_ratio_jun'] = 0
df_X.loc[df_X.credit_amount_jun != 0,'d_by_c_amount_ratio_jun'] = df_X.loc[df_X.credit_amount_jun != 0,'debit_amount_jun']/df_X.loc[df_X.credit_amount_jun != 0,'credit_amount_jun']

###### In the provided code snippet, the target variable cc_cons is being transformed using the natural logarithm (log) in order to directly optimize the Root Mean Squared Log Error (RMSLE) during model training. This transformation is commonly used when the target variable has a skewed distribution or a wide range of values, and the RMSLE metric is commonly used for evaluating models when dealing with such data.

In [53]:
# convert cc_cons to log to directly optimize RMSLE
df_Y=np.log(df_Y+1)

In [54]:
df_Y

0         9.904237
1         9.227591
2         9.686326
3         8.950014
4        11.777318
           ...    
14990     8.357963
14991     9.307467
14992    12.909734
14993    10.268617
14994     9.696033
Name: cc_cons, Length: 14995, dtype: float64

In [56]:
len(df_X.columns)

80

In [57]:
df_X.columns

Index(['account_type', 'gender', 'age', 'cc_cons_apr', 'dc_cons_apr',
       'cc_cons_may', 'dc_cons_may', 'cc_cons_jun', 'dc_cons_jun',
       'cc_count_apr', 'cc_count_may', 'cc_count_jun', 'dc_count_apr',
       'dc_count_may', 'dc_count_jun', 'card_lim', 'personal_loan_active',
       'vehicle_loan_active', 'personal_loan_closed', 'vehicle_loan_closed',
       'investment_1', 'investment_2', 'investment_3', 'investment_4',
       'debit_amount_apr', 'credit_amount_apr', 'debit_count_apr',
       'credit_count_apr', 'max_credit_amount_apr', 'debit_amount_may',
       'credit_amount_may', 'credit_count_may', 'debit_count_may',
       'max_credit_amount_may', 'debit_amount_jun', 'credit_amount_jun',
       'credit_count_jun', 'debit_count_jun', 'max_credit_amount_jun',
       'loan_enq', 'emi_active', 'cc_cons_sum', 'dc_cons_sum', 'cc_count_sum',
       'dc_count_sum', 'cc_cons_avg', 'dc_cons_avg', 'dc_cons_apr_avg',
       'dc_cons_may_avg', 'dc_cons_jun_avg', 'cc_cons_apr_avg',
    

### Train and Test Split

In [58]:
# generate dummy values
df_X=pd.get_dummies(df_X,drop_first=True)

In [61]:
X_train,X_test,Y_train,Y_test=train_test_split(df_X,df_Y,test_size=0.25,random_state=33)

### Modeling

#### Linear Regression

In [67]:
# linear regression with scikit learn
lin_model=LinearRegression()
lin_model.fit(X_train,Y_train)
print('train RMSLE:',round(np.sqrt(mean_squared_error(Y_train,lin_model.predict(X_train)))*100,2))
print('test RMSLE:',round(np.sqrt(mean_squared_error(Y_test,lin_model.predict(X_test)))*100,2));

train RMSLE: 160.82
test RMSLE: 160.73


#### Decision Trees

In [72]:
# hyper parameter tuning of decision tree with randomized search cv
dec_model=DecisionTreeRegressor(random_state=33)
param_grid=dict(max_depth=range(5,20),min_samples_split=range(50,200,10),min_samples_leaf=range(25,100,10),max_leaf_nodes=range(8,32,2),min_impurity_decrease=(0.3,1.0,0.1))
grid=RandomizedSearchCV(dec_model,param_grid,scoring='neg_mean_squared_error',n_jobs=4,cv=5,random_state=33)
grid.fit(X_train,Y_train);

In [73]:
# get best parameters
grid.best_params_

{'min_samples_split': 120,
 'min_samples_leaf': 55,
 'min_impurity_decrease': 1.0,
 'max_leaf_nodes': 18,
 'max_depth': 10}

In [74]:
# take the best model 
dec_model=grid.best_estimator_
print('train RMSLE:',round(np.sqrt(mean_squared_error(Y_train,dec_model.predict(X_train)))*100,2))
print('test RMSLE:',round(np.sqrt(mean_squared_error(Y_test,dec_model.predict(X_test)))*100,2));

train RMSLE: 161.32
test RMSLE: 160.43


#### Neural Network

In [77]:
# hyper parameter tuning of neural networks with randomized search cv
neu_model=MLPRegressor(learning_rate_init=0.01,learning_rate='adaptive',activation='relu',random_state=33)
param_grid={'hidden_layer_sizes':[(100,100,100),(100,75,50,25)],'alpha':range(1,10,1)}
grid=RandomizedSearchCV(neu_model,param_grid,scoring='neg_mean_squared_error',cv=5,n_jobs=-1,random_state=33)
grid.fit(X_train,Y_train);

C:\Users\kimh1\AppData\Roaming\Python\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [78]:
# get best parameters
print(grid.best_params_)

{'hidden_layer_sizes': (100, 75, 50, 25), 'alpha': 6}


In [79]:
# take the best model 
neu_model=grid.best_estimator_
print('train RMSLE:',round(np.sqrt(mean_squared_error(Y_train,neu_model.predict(X_train)))*100,2))
print('test RMSLE:',round(np.sqrt(mean_squared_error(Y_test,neu_model.predict(X_test)))*100,2));

train RMSLE: 162.99
test RMSLE: 162.5


#### Random Forest

In [82]:
# hyper parameter tuning of random forest with randomized search cv
rf_model=RandomForestRegressor(random_state=33)
param_grid=dict(n_estimators=range(10,100,10),max_depth=range(3,20),min_samples_split=range(50,500,20),min_samples_leaf=range(25,75,10),max_leaf_nodes=range(8,32,2))
grid=RandomizedSearchCV(rf_model,param_grid,scoring='neg_mean_squared_error',n_jobs=-1,cv=5,random_state=33)
grid.fit(X_train,Y_train);

In [83]:
# get best score and parameters
grid.best_score_
print(grid.best_params_)

{'n_estimators': 30, 'min_samples_split': 270, 'min_samples_leaf': 55, 'max_leaf_nodes': 26, 'max_depth': 5}


In [84]:
# take the best model 
rf_model=grid.best_estimator_
print('train RMSLE:',round(np.sqrt(mean_squared_error(Y_train,rf_model.predict(X_train)))*100,2))
print('test RMSLE:',round(np.sqrt(mean_squared_error(Y_test,rf_model.predict(X_test)))*100,2));

train RMSLE: 159.4
test RMSLE: 160.4


#### KNN

In [87]:
knn_model=KNeighborsRegressor(n_neighbors=5,weights='distance',algorithm='ball_tree',metric='minkowski',n_jobs=-1)
knn_model.fit(X_train,Y_train);

In [88]:
print('train RMSLE:',round(np.sqrt(mean_squared_error(Y_train,knn_model.predict(X_train)))*100,2))
print('test RMSLE:',round(np.sqrt(mean_squared_error(Y_test,knn_model.predict(X_test)))*100,2));

train RMSLE: 0.0
test RMSLE: 176.65


#### XGBoost

In [91]:
# hyper parameter tuning of xgboost with randomized search cv
xgb_model=XGBRegressor(objective='reg:squarederror',n_jobs=-1)
param_grid={'max_depth':range(5,15,3),'learning_rate':np.arange(0.01,0.3,0.01),'n_estimators':range(100,200,10),'min_child_weight':range(1,15,2),'reg_lambda':range(0,5,1),'subsample':np.arange(60,110,10)/100,'colsample_bylevel':np.arange(60,110,10)/100,'colsample_bytree':np.arange(60,110,10)/100,'colsample_bynode':np.arange(60,110,10)/100,'scale_pos_weight':np.arange(1,15,1)}
grid=RandomizedSearchCV(xgb_model,param_grid,scoring='neg_mean_squared_error',cv=5,random_state=33)
grid.fit(X_train,Y_train);

In [92]:
# get best score and parameters
print(grid.best_params_)

{'subsample': 0.8, 'scale_pos_weight': 1, 'reg_lambda': 3, 'n_estimators': 110, 'min_child_weight': 9, 'max_depth': 5, 'learning_rate': 0.09999999999999999, 'colsample_bytree': 1.0, 'colsample_bynode': 0.6, 'colsample_bylevel': 0.8}


In [93]:
# take the best model 
xgb_model=grid.best_estimator_
print('train RMSLE:',round(np.sqrt(mean_squared_error(Y_train,xgb_model.predict(X_train)))*100,2))
print('test RMSLE:',round(np.sqrt(mean_squared_error(Y_test,xgb_model.predict(X_test)))*100,2))

train RMSLE: 141.11
test RMSLE: 162.52


#### Train on Entire Data

In [94]:
# Random Forest
rf_model=RandomForestRegressor(random_state=33,n_estimators= 50, min_samples_split= 190, min_samples_leaf= 35, max_leaf_nodes= 26, max_depth= 14)
rf_model.fit(df_X,df_Y)
print('Overall RMSLE:',round(np.sqrt(mean_squared_error(df_Y,rf_model.predict(df_X)))*100,2));

Overall RMSLE: 158.06


In [95]:
xgb_model=XGBRegressor(objective='reg:squarederror',subsample= 0.8, scale_pos_weight= 1, reg_lambda= 3, n_estimators= 110, min_child_weight= 9, max_depth= 5, learning_rate= 0.1, colsample_bytree= 1.0,colsample_bynode=0.6,colsample_bylevel=0.8,n_jobs=-1)
xgb_model.fit(df_X,df_Y)
print('Overall RMSLE:',round(np.sqrt(mean_squared_error(df_Y,xgb_model.predict(df_X)))*100,2));

Overall RMSLE: 144.33


#### Model Stacking

In [101]:
lin_model=LinearRegression()
sclf = StackingCVClassifier(classifiers=[rf_model,xgb_model],meta_classifier=lin_model,random_state=33)
print('3-fold cross validation:\n')
for clf, label in zip([rf_model,xgb_model,sclf],['Random Forest','XGBoost','StackingClassifier']):
    scores = np.sqrt(-1*cross_val_score(clf, df_X.values,df_Y.values,cv=5, scoring='neg_mean_squared_error'))*100
    print("Accuracy: %0.2f (+/- %0.2f) [%s]"%(scores.mean(), scores.std(),label))

3-fold cross validation:

Accuracy: 161.22 (+/- 1.76) [Random Forest]
Accuracy: 162.90 (+/- 2.06) [XGBoost]


C:\Users\kimh1\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_validation.py:842: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\kimh1\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_scorer.py", line 136, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "C:\Users\kimh1\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_scorer.py", line 353, in _score
    y_pred = method_caller(estimator, "predict", X)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\kimh1\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
                ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\kimh1\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\_response.py", line 74, in _get_response_values
    classe

KeyboardInterrupt: 

In [102]:
sclf.fit(df_X,df_Y);

KeyboardInterrupt: 

### Final Predictions

#### Load test data

In [ ]:
test=pd.read_csv('Data/test.csv')
len(test)
test.head()

#### Checking the missing value

In [ ]:
# cross check missing values
test.isnull().sum()

#### Treat the missing value

In [ ]:
# treat missing values
test.personal_loan_active.fillna(0,inplace=True)
test.vehicle_loan_active.fillna(0,inplace=True)
test.personal_loan_closed.fillna(0,inplace=True)
test.vehicle_loan_closed.fillna(0,inplace=True)
test.dc_count_apr.fillna(0,inplace=True)
test.dc_count_may.fillna(0,inplace=True)
test.dc_count_jun.fillna(0,inplace=True)
test.loc[test.dc_count_apr==0,'dc_cons_apr']=0
test.loc[test.dc_count_may==0,'dc_cons_may']=0
test.loc[test.dc_count_jun==0,'dc_cons_jun']=0
test.cc_count_apr.fillna(0,inplace=True)
test.cc_count_may.fillna(0,inplace=True)
test.cc_count_jun.fillna(0,inplace=True)
test.loc[test.cc_count_apr==0,'cc_cons_apr']=0
test.loc[test.cc_count_may==0,'cc_cons_may']=0
test.loc[test.cc_count_jun==0,'cc_cons_jun']=0
test.debit_count_apr.fillna(0,inplace=True)
test.debit_count_may.fillna(0,inplace=True)
test.debit_count_jun.fillna(0,inplace=True)
test.loc[test.debit_count_apr==0,'debit_amount_apr']=0
test.loc[test.debit_count_may==0,'debit_amount_may']=0
test.loc[test.debit_count_jun==0,'debit_amount_jun']=0
test.credit_count_apr.fillna(0,inplace=True)
test.credit_count_may.fillna(0,inplace=True)
test.credit_count_jun.fillna(0,inplace=True)
test.loc[test.credit_count_apr==0,'credit_amount_apr']=0
test.loc[test.credit_count_may==0,'credit_amount_may']=0
test.loc[test.credit_count_jun==0,'credit_amount_jun']=0
test.loan_enq.fillna('N',inplace=True)
test.drop(columns=['investment_1','investment_2','investment_3','investment_4'],axis=1,inplace=True)
test.fillna(df.mean(),inplace=True)

#### Feature Engineering

In [ ]:
# change the datatypes of the columns accordingly for generating dummies
categorical_columns=['account_type','gender','region_code','personal_loan_active',
                     'vehicle_loan_active','personal_loan_closed','vehicle_loan_closed','loan_enq']
for c in categorical_columns:
    test[c]=test[c].astype('category');

In [ ]:
test=test.drop(columns=['region_code'])
# create calculated fields
# Total dc and cc cons for 3 months
test['cc_cons_sum']= test['cc_cons_apr'] + test['cc_cons_may'] + test['cc_cons_jun']
test['dc_cons_sum']= test['dc_cons_apr']+ test['dc_cons_may'] + test['dc_cons_jun']
# Total dc and cc count for 3 months
test['cc_count_sum'] = test['cc_count_apr'] + test['cc_count_may'] + test['cc_count_jun']
test['dc_count_sum'] = test['dc_count_apr'] + test['dc_count_may'] + test['dc_count_jun']
# Total dc and cc cons average for 3 months
test['cc_cons_avg'] = 0
test.loc[test.cc_count_sum != 0,'cc_cons_avg'] = test.loc[test.cc_count_sum != 0,'cc_cons_sum']/test.loc[test.cc_count_sum != 0,'cc_count_sum']
test['dc_cons_avg'] = 0
test.loc[test.dc_count_sum != 0,'dc_cons_avg'] = test.loc[test.dc_count_sum != 0,'dc_cons_sum']/test.loc[test.dc_count_sum != 0,'dc_count_sum']
# Average dc cons for each month
test['dc_cons_apr_avg']= 0
test.loc[test.dc_count_apr != 0,'dc_cons_apr_avg'] = test.loc[test.dc_count_apr != 0,'dc_cons_apr']/test.loc[test.dc_count_apr != 0,'dc_count_apr']
test['dc_cons_may_avg'] = 0
test.loc[test.dc_count_may != 0,'dc_cons_may_avg'] = test.loc[test.dc_count_may != 0,'dc_cons_may']/test.loc[test.dc_count_may != 0,'dc_count_may']
test['dc_cons_jun_avg']= 0
test.loc[test.dc_count_jun != 0,'dc_cons_jun_avg'] = test.loc[test.dc_count_jun != 0,'dc_cons_jun']/test.loc[test.dc_count_jun != 0,'dc_count_jun']
# Average cc cons for each month
test['cc_cons_apr_avg'] = 0
test.loc[test.cc_count_apr != 0,'cc_cons_apr_avg'] = test.loc[test.cc_count_apr != 0,'cc_cons_apr']/test.loc[test.cc_count_apr != 0,'cc_count_apr']
test['cc_cons_may_avg'] = 0
test.loc[test.cc_count_may != 0,'cc_cons_may_avg'] = test.loc[test.cc_count_may != 0,'cc_cons_may']/test.loc[test.cc_count_may != 0,'cc_count_may']
test['cc_cons_jun_avg'] = 0
test.loc[test.cc_count_jun != 0,'cc_cons_jun_avg'] = test.loc[test.cc_count_jun != 0,'cc_cons_jun']/test.loc[test.cc_count_jun != 0,'cc_count_jun']
# Total credit and debit amount for 3 months
test['credit_amount_sum'] = test.credit_amount_apr + test.credit_amount_may + test.credit_amount_jun
test['debit_amount_sum'] = test.debit_amount_apr + test.debit_amount_may + test.debit_amount_jun
# Total credit and debit count for 3 months
test['credit_count_sum'] = test.credit_count_apr + test.credit_count_may + test.credit_count_jun
test['debit_count_sum'] = test.debit_count_apr + test.debit_count_may + test.debit_count_jun
# Total credit and debit amount average for 3 months
test['credit_amount_avg'] = 0
test.loc[test.credit_count_sum != 0,'credit_amount_avg'] = test.loc[test.credit_count_sum != 0,'credit_amount_sum']/test.loc[test.credit_count_sum != 0,'credit_count_sum']
test['debit_amount_avg'] = 0
test.loc[test.debit_count_sum != 0,'debit_amount_avg'] = test.loc[test.debit_count_sum != 0,'debit_amount_sum']/test.loc[test.debit_count_sum != 0,'debit_count_sum']
# average debit amount for each month
test['debit_amount_apr_avg']= 0
test.loc[test.debit_count_apr != 0,'debit_amount_apr_avg'] = test.loc[test.debit_count_apr != 0,'debit_amount_apr']/test.loc[test.debit_count_apr != 0,'debit_count_apr']
test['debit_amount_may_avg'] = 0
test.loc[test.debit_count_may != 0,'debit_amount_may_avg'] = test.loc[test.debit_count_may != 0,'debit_amount_may']/test.loc[test.debit_count_may != 0,'debit_count_may']
test['debit_amount_jun_avg'] = 0
test.loc[test.debit_count_jun != 0,'debit_amount_jun_avg'] = test.loc[test.debit_count_jun != 0,'debit_amount_jun']/test.loc[test.debit_count_jun != 0,'debit_count_jun']
# average credit amount for each month
test['credit_amount_apr_avg'] = 0
test.loc[test.credit_count_apr != 0,'credit_amount_apr_avg'] = test.loc[test.credit_count_apr != 0,'credit_amount_apr']/test.loc[test.credit_count_apr != 0,'credit_count_apr']
test['credit_amount_may_avg'] = 0
test.loc[test.credit_count_may != 0,'credit_amount_may_avg'] = test.loc[test.credit_count_may != 0,'credit_amount_may']/test.loc[test.credit_count_may != 0,'credit_count_may']
test['credit_amount_jun_avg'] = 0
test.loc[test.credit_count_jun != 0,'credit_amount_jun_avg'] = test.loc[test.credit_count_jun != 0,'credit_amount_jun']/test.loc[test.credit_count_jun != 0,'credit_count_jun']
# percentage credit amount per month
test['perc_credit_amount_apr'] = 0
test.loc[test.max_credit_amount_apr!= 0,'perc_credit_amount_apr'] = test.loc[test.max_credit_amount_apr!= 0,'credit_amount_apr']/test.loc[test.max_credit_amount_apr!= 0,'max_credit_amount_apr']
test['perc_credit_amount_may'] = 0
test.loc[test.max_credit_amount_may!= 0,'perc_credit_amount_may'] = test.loc[test.max_credit_amount_may!= 0,'credit_amount_may']/test.loc[test.max_credit_amount_may!= 0,'max_credit_amount_may']
test['perc_credit_amount_jun'] = 0
test.loc[test.max_credit_amount_jun!= 0,'perc_credit_amount_jun'] = test.loc[test.max_credit_amount_jun!= 0,'credit_amount_jun']/test.loc[test.max_credit_amount_jun!= 0,'max_credit_amount_jun']
# proportion of debit and credit consumptions per month
test['cc_cons_apr_prop'] = 0
test.loc[test.cc_cons_sum!= 0,'cc_cons_apr_prop'] = test.loc[test.cc_cons_sum!= 0,'cc_cons_apr']/test.loc[test.cc_cons_sum != 0,'cc_cons_sum']
test['cc_cons_may_prop'] = 0
test.loc[test.cc_cons_sum != 0,'cc_cons_may_prop'] = test.loc[test.cc_cons_sum != 0,'cc_cons_may']/test.loc[test.cc_cons_sum != 0,'cc_cons_sum']
test['cc_cons_jun_prop'] = 0
test.loc[test.cc_cons_sum != 0,'cc_cons_jun_prop'] = test.loc[test.cc_cons_sum != 0,'cc_cons_jun']/test.loc[test.cc_cons_sum != 0,'cc_cons_sum']
test['dc_cons_apr_prop'] = 0
test.loc[test.dc_cons_sum != 0,'dc_cons_apr_prop'] = test.loc[test.dc_cons_sum != 0,'dc_cons_apr']/test.loc[test.dc_cons_sum != 0,'dc_cons_sum']
test['dc_cons_may_prop'] = 0
test.loc[test.dc_cons_sum != 0,'dc_cons_may_prop'] = test.loc[test.dc_cons_sum != 0,'dc_cons_may']/test.loc[test.dc_cons_sum != 0,'dc_cons_sum']
test['dc_cons_jun_prop'] = 0
test.loc[test.dc_cons_sum != 0,'dc_cons_jun_prop'] = test.loc[test.dc_cons_sum != 0,'dc_cons_jun']/test.loc[test.dc_cons_sum != 0,'dc_cons_sum']
# proportion of debit and credit amounts per month
test['credit_amount_apr_prop'] = 0
test.loc[test.credit_amount_sum != 0,'credit_amount_apr_prop'] = test.loc[test.credit_amount_sum != 0,'credit_amount_apr']/test.loc[test.credit_amount_sum != 0,'credit_amount_sum']
test['credit_amount_may_prop'] = 0
test.loc[test.credit_amount_sum != 0,'credit_amount_may_prop'] = test.loc[test.credit_amount_sum != 0,'credit_amount_may']/test.loc[test.credit_amount_sum != 0,'credit_amount_sum']
test['credit_amount_jun_prop'] = 0
test.loc[test.credit_amount_sum != 0,'credit_amount_jun_prop'] = test.loc[test.credit_amount_sum != 0,'credit_amount_jun']/test.loc[test.credit_amount_sum != 0,'credit_amount_sum']
test['debit_amount_apr_prop'] = 0
test.loc[test.debit_amount_sum != 0,'debit_amount_apr_prop'] = test.loc[test.debit_amount_sum != 0,'debit_amount_apr']/test.loc[test.debit_amount_sum != 0,'debit_amount_sum']
test['debit_amount_may_prop'] = 0
test.loc[test.debit_amount_sum != 0,'debit_amount_may_prop'] = test.loc[test.debit_amount_sum != 0,'debit_amount_may']/test.loc[test.debit_amount_sum != 0,'debit_amount_sum']
test['debit_amount_jun_prop'] = 0
test.loc[test.debit_amount_sum != 0,'debit_amount_jun_prop'] = test.loc[test.debit_amount_sum != 0,'debit_amount_jun']/test.loc[test.debit_amount_sum != 0,'debit_amount_sum']
# debit by credit cons and amount ratios
test['d_by_c_cons_ratio_apr'] = 0
test.loc[test.cc_cons_apr != 0,'d_by_c_cons_ratio_apr'] = test.loc[test.cc_cons_apr != 0,'dc_cons_apr']/test.loc[test.cc_cons_apr != 0,'cc_cons_apr']
test['d_by_c_cons_ratio_may'] = 0
test.loc[test.cc_cons_may != 0,'d_by_c_cons_ratio_may'] = test.loc[test.cc_cons_may != 0,'dc_cons_may']/test.loc[test.cc_cons_may != 0,'cc_cons_may']
test['d_by_c_cons_ratio_jun'] = 0
test.loc[test.cc_cons_jun != 0,'d_by_c_cons_ratio_jun'] = test.loc[test.cc_cons_jun != 0,'dc_cons_jun']/test.loc[test.cc_cons_jun != 0,'cc_cons_jun']
test['d_by_c_amount_ratio_apr'] = 0
test.loc[test.credit_amount_apr != 0,'d_by_c_amount_ratio_apr'] = test.loc[test.credit_amount_apr != 0,'debit_amount_apr']/test.loc[test.credit_amount_apr != 0,'credit_amount_apr']
test['d_by_c_amount_ratio_may'] = 0
test.loc[test.credit_amount_may != 0,'d_by_c_amount_ratio_may'] = test.loc[test.credit_amount_may != 0,'debit_amount_may']/test.loc[test.credit_amount_may != 0,'credit_amount_may']
test['d_by_c_amount_ratio_jun'] = 0
test.loc[test.credit_amount_jun != 0,'d_by_c_amount_ratio_jun'] = test.loc[test.credit_amount_jun != 0,'debit_amount_jun']/test.loc[test.credit_amount_jun != 0,'credit_amount_jun']
len(test.columns)

#### Prediction

In [ ]:
# generate dummy values
test_X=pd.get_dummies(test.drop(columns=['id']),drop_first=True)
test_Y=sclf.predict(test_X.values)

In [ ]:
sample_submission=pd.concat([test['id'], pd.Series(test_Y)], axis=1)
sample_submission.rename(columns={0:'cc_cons'},inplace=True)
sample_submission['cc_cons']=np.exp(sample_submission['cc_cons'])-1
sample_submission.head()